In [38]:
import pandas as pd
import re
import json


# 待处理的法律文本
def parse_law(text,title):

    # 使用正则表达式按照章的标识符分割文本
    chapters = re.split(r'(第[一二三四五六七八九十]+章)', text)

    current_chapter = ""
    for chunk in chapters:
        # 去除空白字符后，检查当前片段是否为章节标题
        if re.match(r'^第[一二三四五六七八九十]+章', chunk := chunk.strip()):
            current_chapter = chunk  # 更新当前章标题
        else:
            # 查找所有的条目以及对应的内容，确保忽略换行符
            articles = re.findall(
                r'(第[一二三四五六七八九十百]+条)\s+(.*?)(?=(第[一二三四五六七八九十百]+条)|$)',
                chunk, re.DOTALL
            )
            # 遍历条目，构建返回结果
            for article_num, content, _ in articles:
                # 移除内容中的所有换行符，并生成结构化的结果
                clean_content = re.sub(r'\s+', '', content.strip())

                yield {
                    "instruction":"请根据"+"《" +title+"》" + current_chapter + article_num+"回答下列问题",
                    "input": "",
                    "output": clean_content
                }



In [39]:
# 将法律内容拆分为条文，并转换成alpaca格式的数据集
def law_to_alpaca_format(file_path):
    df = pd.read_excel(file_path)
    title = df['title']
    content = df['content']
    processed_law = []
    for row in range(len(df)):
        index = df.index[row]
        current_law = list(parse_law(content[index],title[index]))
        processed_law = processed_law + current_law

    # 输出JSON格式的结构化结果到文件
    with open('output.json', 'w', encoding='utf-8') as out_file:
        json.dump(processed_law, out_file, ensure_ascii=False, indent=2)

    print("处理完成，数据已保存到 output.json 文件中。")


In [40]:
law_to_alpaca_format("predata.xlsx")

处理完成，数据已保存到 output.json 文件中。


In [12]:
import json
import pandas as pd
import os

def split_data(file_path, output_dir="splitted_data", num_splits=10):
    """将 JSON 或 CSV 文件拆分成 num_splits 份"""
    os.makedirs(output_dir, exist_ok=True)

    # 读取数据
    if file_path.endswith(".json"):
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    elif file_path.endswith(".csv"):
        data = pd.read_csv(file_path)
    else:
        raise ValueError("仅支持 JSON 和 CSV 文件")

    # 计算每份数据的大小
    total_size = len(data)
    chunk_size = total_size // num_splits

    # 拆分数据
    for i in range(num_splits):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if i < num_splits - 1 else total_size
        part_data = data[start_idx:end_idx]

        # 生成文件名
        output_file = os.path.join(output_dir, f"chinese_law_{i+1}.{file_path.split('.')[-1]}")

        # 保存数据
        if file_path.endswith(".json"):
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(part_data, f, ensure_ascii=False, indent=2)
        elif file_path.endswith(".csv"):
            part_data.to_csv(output_file, index=False)

        print(f"已生成 {output_file}，包含 {len(part_data)} 条数据")

# 运行示例
split_data("chinese_law.json")  # 替换为你的数据文件名


已生成 splitted_data\chinese_law_1.json，包含 450 条数据
已生成 splitted_data\chinese_law_2.json，包含 450 条数据
已生成 splitted_data\chinese_law_3.json，包含 450 条数据
已生成 splitted_data\chinese_law_4.json，包含 450 条数据
已生成 splitted_data\chinese_law_5.json，包含 450 条数据
已生成 splitted_data\chinese_law_6.json，包含 450 条数据
已生成 splitted_data\chinese_law_7.json，包含 450 条数据
已生成 splitted_data\chinese_law_8.json，包含 450 条数据
已生成 splitted_data\chinese_law_9.json，包含 450 条数据
已生成 splitted_data\chinese_law_10.json，包含 456 条数据


In [6]:
import spacy

nlp = spacy.load("zh_core_web_trf")

def parse_legal_clause(text):
    doc = nlp(text)
    elements = {
        "主体": [ent.text for ent in doc.ents if ent.label_ in ["ORG", "PERSON"]],
        "行为动词": [token.lemma_ for token in doc if token.pos_ == "VERB"],
        "法律后果": [token.text for token in doc if token.text in ["处罚", "解除劳动合同", "赔偿", "承担责任"]],
        "条件状语": [token.text for token in doc if token.dep_ == "advcl"]
    }
    return elements

# 示例条款：《劳动合同法》第三十八条 
clause = "用人单位未及时足额支付劳动报酬的，劳动者可以解除劳动合同"
print(parse_legal_clause(clause))

{'主体': [], '行为动词': ['', '', ''], '法律后果': [], '条件状语': []}


In [7]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("zh_core_web_trf")

# 定义法律后果关键词
legal_consequences = ["解除劳动合同", "赔偿", "承担责任", "罚款", "拘留", "追究刑事责任"]

# 初始化 Matcher
matcher = Matcher(nlp.vocab)
for consequence in legal_consequences:
    pattern = [{"TEXT": consequence}]
    matcher.add(consequence, [pattern])

def parse_legal_clause(text):
    doc = nlp(text)
    
    # 提取主体（组织、个人）
    subjects = [ent.text for ent in doc.ents if ent.label_ in ["ORG", "PERSON"]]
    
    # 提取核心行为动词
    verbs = [token.lemma_ for token in doc if token.pos_ == "VERB" or token.dep_ == "ROOT"]
    
    # 使用规则匹配查找法律后果
    matches = matcher(doc)
    consequences = [doc[start:end].text for match_id, start, end in matches]
    
    # 提取条件状语（状语从句）
    adverbials = [token.text for token in doc if token.dep_ == "advcl"]

    elements = {
        "主体": subjects,
        "行为动词": verbs,
        "法律后果": consequences,
        "条件状语": adverbials
    }
    
    return elements

# 测试案例：《劳动合同法》第三十八条 
clause = "用人单位未及时足额支付劳动报酬的，劳动者可以解除劳动合同"
print(parse_legal_clause(clause))

{'主体': [], '行为动词': ['', '', ''], '法律后果': [], '条件状语': []}


In [ ]:
import pandas as pd
import spacy
from transformers import pipeline

# 加载spaCy的中文模型
nlp = spacy.load("zh_core_web_sm")

# 加载transformers的问答生成模型
question_generator = pipeline("text2text-generation", model="mrm8488/t5-base-finetuned-question-generation-ap")

# 从Excel文件中读取法律条款
def load_legal_articles_from_excel(file_path, sheet_name="条款", column_name="条款内容"):
    try:
        # 使用pandas读取Excel文件
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        if column_name not in df.columns:
            raise ValueError(f"列名 '{column_name}' 不存在于Excel文件中！")
        
        # 提取条款内容
        legal_articles = df[column_name].dropna().tolist()  # 去除空值并转换为列表
        return legal_articles
    except Exception as e:
        print(f"读取Excel文件时出错: {e}")
        return []

# 使用spaCy提取关键信息
def extract_keywords(article):
    doc = nlp(article)
    keywords = {
        "subject": [],  # 主体
        "action": [],   # 行为
        "condition": [],  # 条件
        "responsibility": []  # 责任
    }
    
    for token in doc:
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":  # 名词或专有名词
            keywords["subject"].append(token.text)
        if token.pos_ == "VERB":  # 动词
            keywords["action"].append(token.text)
        if token.dep_ == "advmod" or token.dep_ == "acl":  # 条件修饰
            keywords["condition"].append(token.text)
        if token.dep_ == "dobj" or token.dep_ == "pobj":  # 责任对象
            keywords["responsibility"].append(token.text)
    
    return {k: list(set(v)) for k, v in keywords.items()}  # 去重

# 使用transformers生成问题
def generate_questions_with_transformers(article):
    questions = question_generator(f"generate question: {article}")
    return [q['generated_text'] for q in questions]

# 批量生成问题
def batch_generate_questions(articles):
    all_questions = []
    for article in articles:
        # 提取关键词
        keywords = extract_keywords(article)
        print(f"Extracted Keywords: {keywords}")
        
        # 使用transformers生成问题
        generated_questions = generate_questions_with_transformers(article)
        all_questions.extend(generated_questions)
    
    return all_questions

# 主程序
if __name__ == "__main__":
    # Excel文件路径
    excel_file_path = "output_texts.xlsx"
    
    # 从Excel文件中加载法律条款
    legal_articles = load_legal_articles_from_excel(excel_file_path)
    
    if not legal_articles:
        print("未找到任何法律条款，请检查Excel文件内容！")
    else:
        # 批量生成问题
        questions = batch_generate_questions(legal_articles)
        
        # 输出生成的问题
        print("\nGenerated Questions:")
        for i, q in enumerate(questions, 1):
            print(f"{i}. {q}")